# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
0,0,chonchi,-42.6128,-73.8092,12.56,93,75,1.91,1707451367
1,1,wailua homesteads,22.0669,-159.3780,25.24,72,100,4.63,1707451478
2,2,uturoa,-16.7333,-151.4333,27.81,82,99,6.40,1707451492
3,3,waitangi,-43.9535,-176.5597,16.57,60,18,8.62,1707451478
4,4,rongelap,11.1635,166.8941,26.51,84,100,9.48,1707451518


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    color = "City",
    frame_width = 900,
    frame_height = 650
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

In [4]:
city_data_df.dtypes

City_ID         int64
City           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Date            int64
dtype: object

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df.loc[(city_data_df["Max Temp"] > 21) & (city_data_df["Max Temp"] < 27) & (city_data_df["Wind Speed"] < 4.5) & (city_data_df["Cloudiness"] < 35)]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
133,133,vila velha,-20.3297,-40.2925,24.41,95,20,1.03,1707451541
137,137,champerico,14.3000,-91.9167,25.36,67,16,1.67,1707451719
143,143,lihue,21.9789,-159.3672,26.58,72,22,3.67,1707451197
223,223,sittwe,20.1500,92.9000,24.22,34,1,3.70,1707451887
259,259,tabou,4.4230,-7.3528,25.98,84,7,3.08,1707451895
261,261,muana,-1.5283,-49.2167,24.20,97,34,1.75,1707451896
279,279,manaure,11.7751,-72.4445,24.99,73,16,4.49,1707451899
292,292,cururupu,-1.8283,-44.8683,24.65,94,30,1.76,1707451634
303,303,freetown,8.4840,-13.2299,24.54,89,3,1.71,1707451904
320,320,tolanaro,-25.0319,46.9987,23.04,94,20,4.12,1707451485


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
copy_df = ideal_weather_df.copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df = copy_df[["City", "Lat", "Lng", "Humidity"]]
hotel_df["Hotel Name"] = " "

# Display sample data
hotel_df

/var/folders/sm/718jcs250_gclfbkg39pptbw0000gn/T/ipykernel_46124/671350292.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = " "


,City,Lat,Lng,Humidity,Hotel Name
133,vila velha,-20.3297,-40.2925,95,
137,champerico,14.3000,-91.9167,67,
143,lihue,21.9789,-159.3672,72,
223,sittwe,20.1500,92.9000,34,
259,tabou,4.4230,-7.3528,84,
261,muana,-1.5283,-49.2167,97,
279,manaure,11.7751,-72.4445,73,
292,cururupu,-1.8283,-44.8683,94,
303,freetown,8.4840,-13.2299,89,
320,tolanaro,-25.0319,46.9987,94,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey": geoapify_key,
    "categories": "accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = radius
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
vila velha - nearest hotel: Hotel Prainha
champerico - nearest hotel: Hotel y Restaurante El Submarino
lihue - nearest hotel: Kauai Palms
sittwe - nearest hotel: Yuzana Aung Motel 1
tabou - nearest hotel: hôtel le rochet
muana - nearest hotel: Hotel Jarumã
manaure - nearest hotel: hotel palaima
cururupu - nearest hotel: No hotel found
freetown - nearest hotel: Formal Leona hotel
tolanaro - nearest hotel: Hôtel Mahavokey
kongolo - nearest hotel: No hotel found
barra velha - nearest hotel: Candeias
uribia - nearest hotel: hotel palaima
castillos - nearest hotel: A mi gente
balimila - nearest hotel: Bala Lodge
richards bay - nearest hotel: SUN1 Richards Bay
zungeru - nearest hotel: No hotel found
acapulco de juarez - nearest hotel: Hotel del Valle
san mateo del mar - nearest hotel: Gran Hotel Don Nacho
mongoumba - nearest hotel: Hôtel Wilo et Frères
kolondieba - nearest hotel: No hotel found
raja - nearest hotel: No hotel found
boali - nearest hotel: No hotel found
c

,City,Lat,Lng,Humidity,Hotel Name
133,vila velha,-20.3297,-40.2925,95,Hotel Prainha
137,champerico,14.3000,-91.9167,67,Hotel y Restaurante El Submarino
143,lihue,21.9789,-159.3672,72,Kauai Palms
223,sittwe,20.1500,92.9000,34,Yuzana Aung Motel 1
259,tabou,4.4230,-7.3528,84,hôtel le rochet
261,muana,-1.5283,-49.2167,97,Hotel Jarumã
279,manaure,11.7751,-72.4445,73,hotel palaima
292,cururupu,-1.8283,-44.8683,94,No hotel found
303,freetown,8.4840,-13.2299,89,Formal Leona hotel
320,tolanaro,-25.0319,46.9987,94,Hôtel Mahavokey


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
map_plot2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    color = "City",
    frame_width = 900,
    frame_height = 650,
    hover_cols=["Hotel Name"]
)

# Display the map
map_plot2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name)